# Предсказание дефолта по кредиту

Многие люди берут кредит в банке, некоторые не отдают. Мы не знаем, что значит каждый из 5 предикторов, поэтому не будем описывать каждый в подробности. 
Задача: построить две модели, предсказывающие дефолт потенциального заемщика по кредиту.

Задача предсказания дефолта - задача классификации, а результат работы модели для конкретного клиента - предсказать произойдет дефолт или нет.
В этой тетради мы рассмотрим вопрос измерения качества работы обученных
классификаторов. На примерах мы опишем различные метрики качества и способы
их подсчёта.

**Бизнес-постановка задачи** 

Банк по анкетным данным оценивает вероятность того, что для конкретного клиента произойдет дефолт.
Применение модели:
* банк хочет выдавать кредиты только хорошим заемщикам, которые отдадут кредит.

**Постановка задачи анализа данных** 

Целью данной задачи является построение модели *классификации дефолтов*: на вход модель будет принимать данные о клиенте, а на выходе она должна работать в двух режимах:
* выдавать вероятность дефолта для данного клиента,
* выдавать правильный с точки зрения модели класс клиента (есть у него дефолт или нет).

Обучить модель по имеющимся данным тестового задания.

**Обзор доступных данных**

В выборке 5000 наблюдений и 6 переменных, одна из которых - целевая. 

Выборка была разбита на две части для обучения и для тестирования модели.
В обучающей выборке 5000 клиентов, в тестовой выборке - ?

## 1. Загрузить данные для обучения

**Шаг 1.1. Загружаем библиотеки**  

In [53]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style('whitegrid')

import warnings
warnings.filterwarnings("ignore")
from sklearn import linear_model, ensemble

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve,  auc # метрики качества
from sklearn.metrics import confusion_matrix # метрики качества

**Шаг 1.2. Загрузим данные**

Для решения задачи мы будем использовать данные. Они состоят из двух частей: часть для обучения и часть для тестирования модели. Загружаем данные с помощие команды !wget. Для того, чтобы игнорировать сообщения в процессе загрузки используем магическую команду %%capture в первой строке.

In [2]:
!git clone https://github.com/IlonaInch/Default_prediction.git

Cloning into 'Default_prediction'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 17 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (17/17), done.



Наши данные в формате `csv` - delimiter separated values, значения, разделенные точкой с запятой. Чтобы загрузить  данные, используем функцию **`pd.read_csv()`** из библиотеки `pandas`.

In [3]:
df = pd.read_csv('/content/Default_prediction/csv_default_1.csv', sep= ';')
df

,No,predictor 1,predictor 2,predictor 3,predictor 4,predictor 5,target
0,1,12423,710,"441,5",5,4631,1
1,2,13983,1106,NaN,3,4256,1
2,3,15520,1276,"391,5",1,4397,0
3,4,29535,1106,NaN,1,3772,1
4,5,26154,976,315,5,2445,1
...,...,...,...,...,...,...,...
9995,9 996,17220,1450,"421,5",3,1759,0
9996,9 997,6173,1100,NaN,1,4521,0
9997,9 998,41944,1122,311,13,4087,0
9998,9 999,28381,1314,182,1,4439,1


**Шаг 1.3. Посмотрим общую статистику по данным**


In [4]:
df.head(5)

,No,predictor 1,predictor 2,predictor 3,predictor 4,predictor 5,target
0,1,12423,710,"441,5",5,4631,1
1,2,13983,1106,NaN,3,4256,1
2,3,15520,1276,"391,5",1,4397,0
3,4,29535,1106,NaN,1,3772,1
4,5,26154,976,315,5,2445,1


In [5]:
df.shape

(10000, 7)

# 2. Обработать данные перед обучением модели

**Шаг 2.1. Проверяем данные на наличие пропусков и типов переменных**

In [29]:
df.info()
#видим, что предикторы 3 и 5 типа "объект"

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3120 entries, 4 to 9999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   No           3120 non-null   object 
 1   predictor 1  3120 non-null   int64  
 2   predictor 2  3120 non-null   int64  
 3   predictor 3  3120 non-null   float64
 4   predictor 4  3120 non-null   int64  
 5   predictor 5  3120 non-null   float64
 6   target       3120 non-null   int64  
dtypes: float64(2), int64(4), object(1)
memory usage: 195.0+ KB


In [30]:
# проверка количества уникальных значений
df.nunique()

No             3120
predictor 1    2997
predictor 2     365
predictor 3     296
predictor 4      19
predictor 5    2325
target            2
dtype: int64

In [31]:
# наличие NaN значений
df.isna().sum()

No             0
predictor 1    0
predictor 2    0
predictor 3    0
predictor 4    0
predictor 5    0
target         0
dtype: int64

In [32]:
# как мы видим, в предикторе 3 есть NAN значения в количестве 1913

In [33]:
df.dropna(inplace=True)
df.shape
# удалили значения NAN и смотрим shape, сколько участвуют в эксперименте

(3120, 7)

In [34]:
df[["predictor 3", "predictor 5"]] = df[["predictor 3", "predictor 5"]].apply(pd.to_numeric, errors = 'coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3120 entries, 4 to 9999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   No           3120 non-null   object 
 1   predictor 1  3120 non-null   int64  
 2   predictor 2  3120 non-null   int64  
 3   predictor 3  3120 non-null   float64
 4   predictor 4  3120 non-null   int64  
 5   predictor 5  3120 non-null   float64
 6   target       3120 non-null   int64  
dtypes: float64(2), int64(4), object(1)
memory usage: 195.0+ KB



**Шаг 2.2. Заполнение пропусков**

Рассчитаем средние значения признаков в обучающей выборке, и заполним полученными
числами пропуски как в **тестовом наборе** данных, так и в **самой обучающей выборке**.




In [35]:
df['target'].value_counts()
#видим, что кол-во хороших заемщиков = 2489, а плохих = 631

0    2489
1     631
Name: target, dtype: int64

In [36]:
training_values = df['target']
training_values.shape

(3120,)

In [37]:
training_points = df.drop(['target', 'No'], axis=1)
training_points.shape

(3120, 5)

In [38]:

logistic_regression_model = linear_model.LogisticRegression(C=2) # создаем модель
random_forest_model = ensemble.RandomForestClassifier(n_estimators=50)

In [39]:
mean = df.mean()
mean

predictor 1    21672.439103
predictor 2     1122.437821
predictor 3      367.647756
predictor 4        3.832692
predictor 5     2514.570192
target             0.202244
dtype: float64

**Шаг 2.3. Работаем с целевой переменной**

Мы должны прогнозировать дефолт, поэтому целевая переменная - это наличие дефолта. 

Отделим входные переменные от выходной (целевой), чтобы можно было построить модель предсказания целевой переменной по входным.

In [40]:
df.dropna(inplace=True)
train_data = df.drop(['target', 'No'], axis=1)

train_data.shape
train_data
#удалили ненужные столбцы Номер и целевой target

,predictor 1,predictor 2,predictor 3,predictor 4,predictor 5
4,26154,976,315.0,5,2445.0
14,17262,1106,169.0,3,2886.0
16,13577,1096,401.0,5,4794.0
19,6233,1060,419.0,5,1401.0
20,13382,1040,304.0,7,2923.0
...,...,...,...,...,...
9993,12802,968,351.0,3,2548.0
9994,796,1302,410.0,5,4286.0
9997,41944,1122,311.0,13,4087.0
9998,28381,1314,182.0,1,4439.0


In [41]:
test_data = df['target']
test_data.shape

(3120,)

##   3. Обучить модель на обучающей выборке

**Шаг 3.1. Выбираем метод, который будем использовать**

Воспользуемся двумя методами для построения моделей классификации и сравним их между собой:
* Логистическая регрессия *logistic regression*
* Случайный лес *random forest*


In [43]:
logistic_regression_model = linear_model.LogisticRegression() # создаем модель

In [44]:
random_forest_model = ensemble.RandomForestClassifier(n_estimators=50)

**Шаг 3.2. Обучить модель**


In [45]:
train_data = train_data.astype(int) # to make sure that the data is actually in int format.
test_data = test_data.astype(int)

In [46]:

X_train, X_test, Y_train, Y_test = train_test_split(train_data, test_data, test_size=0.25, random_state=7)
#разбивка данных на обучение 70% и тест 30%
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(2340, 5) (780, 5) (2340,) (780,)


In [48]:
lr_clf = LogisticRegression()
lr_clf.fit(X_train, Y_train, 0.3)

LogisticRegression()

In [49]:

lr_pred = lr_clf.predict(X_train)
train_score = accuracy_score(Y_train, lr_pred) * 100
#R_squared1 = r2_score(lr_pred, y_train)
print(f"Train accuracy score: {train_score:.2f}%,")

lr_pred = lr_clf.predict(X_test)
test_score = accuracy_score(Y_test, lr_pred) * 100
#R_squared = r2_score(lr_pred, y_test)
print(f"Test accuracy score: {test_score:.2f}%,")

Train accuracy score: 81.32%,
Test accuracy score: 77.44%,


Точно также построим модель случайного леса

In [50]:

clfRFC = RandomForestClassifier()
clfRFC.fit(X_train, Y_train, 0.3)


RandomForestClassifier()

In [51]:

clfRFC_predict = clfRFC.predict(X_train)
train_score = accuracy_score(Y_train, clfRFC_predict) * 100
#R_squared1 = r2_score(lr_pred, y_train)
print(f"Train accuracy score: {train_score:.2f}%,")

clfRFC_predict = clfRFC.predict(X_test)
test_score = accuracy_score(Y_test, clfRFC_predict) * 100
#R_squared = r2_score(lr_pred, y_test)
print(f"Test accuracy score: {test_score:.2f}%,")

Train accuracy score: 100.00%,
Test accuracy score: 77.56%,


* Мы получили две обученные модели. Обе модели показали высокую точность

### Шаг 5.2. Матрица ошибок модели классификации

ROC кривая классификатора